In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab

    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/departamentos-en-venta-2016.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/departamentos-en-venta-2016.csv?token=AA4GFHLZYO72DPTOKLF4LH26V3KTI
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Intro

En esta parte vimos algunos métodos de Pandas para calcular medidas por grupos.

Para los ejercicios usaremos un dataset de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016 que disponibiliza el GCBA.

La idea de esta práctica es repasar algunos temas que vimos en clases anteriores, y ejercitar los distintos métodos que vimos en ésta y vamos a volver a usar en la clase presencial.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [ ]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [ ]:
data_location = "../Data/departamentos-en-venta-2016.csv"
data = pd.read_csv(data_location, sep=";")
print(data.shape)
print(data.dtypes)
data.head(3)

## Ejercicio 2  - Limpieza

Vamos a crear una columan nueva (PRECIOTEXT_CLEAN) de tipo numérico, que tenga los valores en dolares que leemos en el campo PRECIOTEXT, o NaN si esos valores están expresados en pesos.

Para eso vamos a:

1) Crear un objeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)


In [ ]:
import re

# reemplazo U$S por vacío
dolares_pattern = "U\$S\s*"
dolares_regex = re.compile(dolares_pattern)

# reemplazo el punto decimal por vacío
decimal_pattern = "\."
decimal_regex = re.compile(decimal_pattern)

sin_dolar = data.PRECIOTEXT.apply(lambda x: x if x is np.NaN else dolares_regex.sub("", x))

sin_dolar_sin_punto = sin_dolar.apply(lambda x: x if x is np.NaN else decimal_regex.sub("", x))

#type(sin_dolar_sin_punto)

In [ ]:
# armo una máscara con los registros que tienen el valor de PRECIOTEXT en $
pesos_pattern = "\$"
pesos_regex = re.compile(pesos_pattern)

matches_pesos = sin_dolar_sin_punto.apply(lambda x: x if x is np.NaN else pesos_regex.match(x))
pesos_mask = matches_pesos.notnull()
#pesos_mask.sum()

#veo que egfectivamente esos son los valores con $
print(sin_dolar_sin_punto[pesos_mask])

# asigno np.NaN a esos valores
sin_dolar_sin_punto[pesos_mask] = np.NaN


In [ ]:
sin_dolar_sin_punto_sin_pesos_num  = sin_dolar_sin_punto.astype(float)
data["PRECIOTEXT_CLEAN"] = sin_dolar_sin_punto_sin_pesos_num


## Ejercicio 3  - Precio por metro cuadrado en dolares

Vamos a crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

In [ ]:
data["PRECIOTEXTM"] = (data["PRECIOTEXT_CLEAN"] / data["M2"]).round(2)
data.head(3)


## Ejercicio 4  - Precio promedio del metro cuadrado en dolares por barrio

Calculemos el precio promedio del metro cuadrado en dolares por barrio usando groupby y pivot_table

¿Cuáles son los cinco barrios más caros?

In [ ]:
precio_m2_prom_barrio = data.groupby("BARRIO")["PRECIOTEXTM"].mean().round(2)
#print(precio_m2_prom_barrio)
#type(precio_m2_prom_barrio)
precio_m2_prom_barrio_sort = precio_m2_prom_barrio.sort_values(ascending = False)
precio_m2_prom_barrio_top5 = precio_m2_prom_barrio_sort[0:5]
precio_m2_prom_barrio_top5

In [ ]:
precio_m2_prom_barrio = data.pivot_table(index = 'BARRIO', aggfunc={'PRECIOTEXTM':'mean'}) 
precio_m2_prom_barrio_sort = precio_m2_prom_barrio.sort_values(by = "PRECIOTEXTM", ascending = False)
precio_m2_prom_barrio_top5 = precio_m2_prom_barrio_sort[0:5]
precio_m2_prom_barrio_top5    


## Ejercicio 5  - groupby por campos numéricos (decilos)

Armemos los decilos de superficie de data usando el método `qcut` de numpy

Esto asocia una categoría a cada registro de data, donde el valor de esa categoría indica a qué decilo pertenece ese registro

Usando el método `groupby` calculemos la mediana del precio por m2 en dolares para cada uno de los decilos de superficie.

In [ ]:
q_superficie = pd.qcut(data.M2, 10)
q_superficie

In [ ]:
data.groupby(q_superficie)["PRECIOTEXTM"].median()

## Ejercicio 6  - groupby por campos de tipo string

Usando el método `groupby` calculemos la mediana del precio por m2 en dolares para cada uno de los barrios de la ciudad.

In [ ]:
data.groupby("BARRIO")["PRECIOTEXTM"].median()

## Ejercicio 7  - groupby por condiciones custom

Definamos una máscara booleana que sea True para los registros cuyo precio en dolares es hasta 150.000

Agrupemos por este criterio, y calculemos el valor de la mediana de superficie para estos dos grupos (precio <= 150.000 dolares y precio > 150.000 dolares)

In [ ]:
mask_price = data.PRECIOTEXT_CLEAN <= 150000

data.groupby(mask_price)["M2"].median()